In [ ]:
fnames = []
target_dir = test_dir
dnames = [join(target_dir, dname) for dname in listdir(target_dir)]

for dname in dnames:
    
    fnames =([join(dname, fname) for fname in listdir(dname)])
    
    cur_class = dname.split('\\')[-1]
    print(cur_class, end=':- ')

    NO_OF_FIG = 15
    correct_predict_count = 0
    high_accuracy_count = 0
    
    max_indices = []
    wrong_class_list = []
    
    for index, img_path in enumerate(fnames):
          
        img = image.load_img(img_path, target_size=(IMAGE_SIZE, IMAGE_SIZE)) 
        
        img_tensor = image.img_to_array(img)
        img_tensor = img_tensor.reshape((1,) + img_tensor.shape)
        img_tensor /= 255.
        prediction = model.predict(img_tensor)[0]
        
        max_val = np.amax(prediction)
        max_index = np.argmax(prediction)
        
        pred_class = CLASSES_TO_CLASSIFY[max_index]
        
        if pred_class == cur_class:         
            correct_predict_count +=1
        else:
            wrong_class_list.append(pred_class)
            
        if max_val > 0.8:
            high_accuracy_count += 1
          
    print('Accuracy: ' + str(correct_predict_count / len(fnames) * 100) + ' , High count: ' + str(high_accuracy_count))
    print('\n' + str(wrong_class_list) + '\n')
    

In [ ]:
fnames = []
target_dir = test_dir
dnames = [join(target_dir, dname) for dname in listdir(target_dir)]

for dname in dnames:
    
    fnames =([join(dname, fname) for fname in listdir(dname)])
    
    cur_class = dname.split('\\')[-1]
    print(cur_class, end=':- ')

    correct_predict_count = 0
    high_accuracy_count = 0
    
    for index, img_path in enumerate(fnames):
          
        img = image.load_img(img_path, target_size=(IMAGE_SIZE, IMAGE_SIZE)) 
        
        img_tensor = image.img_to_array(img)
        img_tensor = img_tensor.reshape((1,) + img_tensor.shape)
        img_tensor /= 255.
        prediction = model.predict(img_tensor)[0]
        
        if cur_class == 'others':
            if prediction[0] <= 0.5:
                correct_predict_count +=1
        else:
            if prediction[0] > 0.5:
                correct_predict_count +=1

          
    print('Accuracy: ' + str(correct_predict_count / len(fnames) * 100) + ' , High count: ' + str(high_accuracy_count))
    

Slice images

In [ ]:
with Image.open(MAP_FILE_PATH) as im:
    imgwidth, imgheight = im.size
    SLICE_WIDTH, SLICE_HEIGHT = 256, 256
    idx = 0
    start_idx = 14000
    delete_dir_files_helper(TEMP_SLICED_SUBDIR)
    
    for i in range(0, imgheight - SLICE_HEIGHT, SLICE_HEIGHT):
        for j in range(0, imgwidth - SLICE_WIDTH, SLICE_WIDTH):
            box = (j, i, j + SLICE_WIDTH, i + SLICE_HEIGHT)
            try:
                cropped_image = im.crop(box)
                cropped_image.save(join(TEMP_SLICED_SUBDIR, 'image_{}.jpg'.format(start_idx + idx)), "JPEG", quality=100, optimize=True)
                idx += 1
            except:
                traceback.print_exc()
                
                pass
    print('Saved ' + str(idx) + ' slices in ' + TEMP_SLICED_SUBDIR)

predict using data generator

In [ ]:
#     test_datagen = ImageDataGenerator(rescale=1./255)
#     test_generator = test_datagen.flow_from_directory(
#         TEMP_SLICED_DIR,
#         target_size=(IMAGE_SIZE, IMAGE_SIZE),
#         batch_size=1,
#         save_prefix='slices',
#         save_format='jpeg',
#         shuffle=False,
#         save_to_dir=AUG_DIR,
#         class_mode=None)
    
#     predictions = model.predict_generator(test_generator, no_of_samples , max_queue_size=1, workers=5, use_multiprocessing=False, verbose=1)
#     print(predictions.shape)
#     print(test_generator.filenames)

In [ ]:
def predict_class_overlay(prediction):
    
    if prediction > 0.4:
        return (0, 255, 0, 100)
    else:
        return (0, 255, 0, 0)


with Image.open(MAP_FILE_PATH) as map_image:
       
    no_of_samples = len(listdir(TEMP_SLICED_SUBDIR))
    MODEL_PATH =  join(MODEL_DIR, 'roads_binary_classifer_model.h5');
    
    imgwidth, imgheight = map_image.size
    idx = 0
    
    model = load_model(MODEL_PATH)
    model.compile(loss='binary_crossentropy',optimizer=optimizers.RMSprop(lr=1e-4),metrics=['acc'])

    test_datagen = ImageDataGenerator(rescale=1./255)
    test_generator = test_datagen.flow_from_directory(
        TEMP_SLICED_DIR,
        target_size=(SLICE_WIDTH, SLICE_HEIGHT),
        batch_size=1,
        save_format='jpeg',
        save_to_dir=AUG_DIR,
        shuffle=False,
        class_mode=None)

    predictions = model.predict_generator(test_generator, no_of_samples, max_queue_size=10, workers=8, use_multiprocessing=False, verbose=0)
    predictions = np.asarray(predictions).flatten()
  
    for i in range(0, imgheight , SLICE_HEIGHT):
        x_offset = 0
        for j in range(0, imgwidth , SLICE_WIDTH):
            box = (j, i, j + SLICE_WIDTH, i + SLICE_HEIGHT)
            try:
                
                overlay_color = predict_class_overlay(predictions[idx])
                overlay = Image.new('RGBA', ( SLICE_WIDTH, SLICE_HEIGHT), overlay_color)  
                map_image.paste(overlay, (j, i), overlay)
            except:
                traceback.print_exc()
                pass
            idx +=1
      
    map_image.save(join(OUTPUT_DIR, 'binary_class_map_output.jpg'), "JPEG", quality=100, optimize=True)
    plt.figure(figsize=(30, 30)) 
    plt.imshow(map_image)
    plt.axis('off')